In [5]:
from pyomo.environ import *
from itertools import combinations
import pandas as pd

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [6]:
dem = pd.read_excel('C:/Jupyter/TESIS/instancias/demanda_n7_k150_t60.xlsx')
preco = pd.read_excel('C:/Jupyter/TESIS/instancias/preco_n7_k150_t60.xlsx')
cap = 700 #capacidade do trem
n = 7 #numero de estaciones
kk = 150 #numero de classes
tt = 60 #numero de períodos
# estacoes = ['E1','E2', 'E3','E4']
estacoes = [1,2,3,4,5,6,7]
combinaciones = list(combinations(estacoes, 2))
oridest = [(inicio, fin) for inicio, fin in combinaciones if estacoes.index(inicio) < estacoes.index(fin)]
legs = [(inicio, fin) for inicio, fin in combinaciones if estacoes.index(fin) == estacoes.index(inicio)+1]

In [7]:
dem_cor = {}
for i in range(dem.shape[0]): #dem.iloc[0:12,:]
    dem_cor[(dem.iloc[i,0], dem.iloc[i,1], dem.iloc[i,2], dem.iloc[i,3])] = dem.iloc[i,4]

preco_cor = {}
for i in range(preco.shape[0]):
    preco_cor[(preco.iloc[i,0], preco.iloc[i,1], preco.iloc[i,2])] = preco.iloc[i,3]

In [ ]:
# instanciar o modelo
model = AbstractModel()

# Escalares
model.n = Param(initialize = n)
model.k = Param(initialize = kk)
model.t = Param(initialize = tt)
model.Q = Param(initialize = cap)

# conjuntos
model.ES = Set(initialize = estacoes)
model.OD = Set(dimen = 2, initialize = oridest)
model.L = Set(dimen = 2, initialize = legs)
model.K = RangeSet(1, model.k)
model.T = RangeSet(1, model.t)

#parâmetros indexados
model.P = Param(model.OD, model.K, initialize = preco_cor)
model.d = Param(model.OD, model.K, model.T, initialize = dem_cor)

# variáveis de decisão
model.X = Var(model.OD, model.K, model.T, within = NonNegativeIntegers)
model.Y = Var(model.OD, model.K, model.T, within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum( model.P[i,j,k]*((model.X[i,j,k,t])) for i,j in model.OD for k in model.K for t in model.T)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def rest1(model):
    return sum(model.X[i,j,k,t]  for i,j in model.OD for k in model.K for t in model.T if i==1) <= model.Q
model.ConstrainRest1 = Constraint(rule = rest1)

def rest2(model, il, jl):
    if (il, jl) != (1,2):
        return sum(model.X[i,j,k,t]  for i,j in model.OD for k in model.K for t in model.T if j==jl) <= model.Q
    else:
        return Constraint.Skip
model.ConstrainRest2 = Constraint(model.L, rule = rest2)

def authorization(model, i, j, k, t):
    return model.Y[i,j,k,t] >= model.X[i,j,k,t]
model.ConstrainAuthorization = Constraint(model.OD, model.K, model.T, rule = authorization)

def classe(model, i, j, k, t):
    if k < model.k:
        return model.Y[i,j,k,t] <= model.Y[i,j,k+1,t]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.OD, model.K, model.T, rule = classe)

def assignment(model, i, j, k, t):
    return model.X[i,j,k,t] <= model.d[i,j,k,t]
model.ConstrainAssignment = Constraint(model.OD, model.K, model.T,rule = assignment)

instance = model.create_instance()
# instance.pprint()

opt = SolverFactory('cplex')
results = opt.solve(instance, tee=True)
print(results)

In [ ]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i in range(1,n+1):
    for j in range(1,n+1):
        for k in range(1,kk+1):
            for t in range(1,tt+1):
                if i<j:
                    # print(i,j,k, ' - ',value(instance.Y[i,j,k]), ' - ', value(instance.X[i,j,k]) )
                    lista.append([str(i) + '-' + str(j), k, t, value(instance.P[i,j,k]), value(instance.d[i,j,k,t]) , value(instance.X[i,j,k,t]), value(instance.Y[i,j,k,t]) ])

a = pd.DataFrame(lista, columns=['o-d','classe','Periodo','preco','demanda','Assignments','Authorizations'])
print(a)

In [11]:
a = pd.DataFrame(lista, columns=['o-d','classe','Periodo','preco','demanda','Assignments','Authorizations'])
a.to_excel('wwww4.xlsx', index=False)